In [251]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.pyplot as plt
from scipy.interpolate import griddata

#Inputs to later make them as arguments
start_year = 1975
end_year = 2020
duration_gap = 5
num_slots = 2020 - 1975 // duration_gap
#month_to_analyse = [3,4,5,6]

cols = ['STATION', 'NAME', 'LATITUDE', 'LONGITUDE', 'DATE', 'MAX', 'MIN', 'PRCP', 'TEMP']
df = pd.read_csv(r'./3849979.csv',usecols=cols)
df['DATE'] = pd.to_datetime(df['DATE'])
df['YEAR'] = df['DATE'].dt.year
df['MONTH'] = df['DATE'].dt.month

station_grps = df.groupby('STATION')
station_names = df['STATION'].unique()
lat = np.zeros(len(station_names))
longt = np.zeros(len(station_names))
station_count = 0
tavg_diff = np.zeros((len(station_names), num_slots, 12))

ref = pd.DataFrame()

def subtract_df_col(a,b,c):
    res = []
    for i in range(0, len(a)):
        if (a[c][i] == 0 or pd.isna(a[c][i]) or b[c][i] == 0 or pd.isna(b[c][i])):
            res.append(0)
        else:
            res.append(a[c][i] - b[c][i])

    return res

for station in station_names:
    kgroup = station_grps.get_group(station)
    #print(kgroup.head())
    lat[station_count] = kgroup['LATITUDE'].reset_index(drop=True)[0]
    
    longt[station_count] = kgroup['LONGITUDE'].reset_index(drop=True)[0]

    if (ref.empty):
        result = kgroup.groupby(['YEAR', 'MONTH','STATION'])['TEMP'].mean().reset_index()
        result = result[result['YEAR'] == 1975]
        #print(type(result))
        ref = result.sort_values('MONTH').reset_index(drop=True)
        #print(result)

    for i in range(start_year + 5, end_year, duration_gap):
        tmp = kgroup.groupby(['YEAR', 'MONTH','STATION'])['TEMP'].mean().reset_index()
        tmp = tmp[tmp['YEAR'] == i]
        tmp = tmp.sort_values('MONTH').reset_index(drop=True)
        #print(tmp)
        #print(ref)
        dif_list = subtract_df_col(tmp, ref, 'TEMP')
        if (len(dif_list) < 12):
            for i in range(len(dif_list), 12):
                dif_list.append(0)
        tavg_diff[station_count][(i - start_year - 5) // 5] = np.array(dif_list)
    
    station_count += 1



In [259]:

fig, axes = plt.subplots(12, 1, figsize=(100, 100))

axes = axes.flatten()

arr_transposed = tavg_diff.transpose(1, 2, 0)

month_data = [val for val in range(0,12)]
year = 2000
for i, ax in enumerate(month_data):
    
    ax = axes[month_data]

    ax = plt.subplot(12, 1, i + 1, projection=ccrs.PlateCarree())
    
    # San Diego co-ordinates
    ax.set_extent([-117.462, -116.866, 32.418 , 33.014 ], crs=ccrs.PlateCarree())
    ax.add_feature(cfeature.COASTLINE)
    ax.add_feature(cfeature.BORDERS, linestyle=":")
    ax.add_feature(cfeature.STATES, edgecolor="gray")

    sc = ax.scatter(
        longt,
        lat,
        c=arr_transposed[(year - start_year - 5) // 5][i],
        cmap="coolwarm",
        s=10000,
        edgecolor="k",
        alpha=0.6,
        transform=ccrs.PlateCarree(),
    )
    ax.set_title(f"{month_data[i]} Temperature", fontsize=10)

cbar_ax = fig.add_axes([0.92, 0.15, 0.02, 0.7])
fig.colorbar(sc, cax=cbar_ax, label="Temperature (°C)")

fig.suptitle("Month-wise Temperature Distribution Across North America", fontsize=16)

plt.tight_layout(rect=[0, 0, 0.9, 0.95])

plt.show()

    







C:\Users\Chandan\AppData\Local\Temp\ipykernel_23028\1681905440.py:38: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout(rect=[0, 0, 0.9, 0.95])


In [255]:
#Alternative way of temperature interpolation fo fill up the map
from scipy.interpolate import griddata
latitudes = np.linspace(33.014, 32.418, 100)
longitudes = np.linspace(-117.462, -116.866, 100)

lon_grid, lat_grid = np.meshgrid(longitudes, latitudes)

fig, axes = plt.subplots(12, 1, figsize=(100, 100))
axes = axes.flatten()

month_data = [val for val in range(0,12)]
year = 2000
for i, ax in enumerate(month_data):
    
    ax = axes[month_data]
    
    ax = plt.subplot(12, 1, i + 1, projection=ccrs.PlateCarree())
    
    ax.set_extent([-117.462, -116.866, 32.418 , 33.014 ], crs=ccrs.PlateCarree())
    ax.add_feature(cfeature.COASTLINE)
    ax.add_feature(cfeature.BORDERS, linestyle=":")
    ax.add_feature(cfeature.STATES, edgecolor="gray")
    
    temperatures = arr_transposed[(year - start_year - 5) // 5][i]
    temp_grid = griddata(
        (longt, lat), temperatures, (lon_grid, lat_grid), method="cubic"
    )
    
    im = ax.imshow(
        temp_grid, origin="lower", extent=(-117.462, -116.866, 32.418 , 33.014 ), cmap="coolwarm", alpha=0.7
    )    
    ax.set_title(f"Temperature in {month_data}", fontsize=12)
    
cbar_ax = fig.add_axes([0.92, 0.15, 0.02, 0.7])
fig.colorbar(sc, cax=cbar_ax, label="Temperature (°C)")
plt.tight_layout()
plt.show()

C:\Users\Chandan\AppData\Local\Temp\ipykernel_23028\3821981172.py:36: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()
